# Pinterest ETL pipeline (spark alone)

### Import packages

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from cassandra.cluster import Cluster
import multiprocessing
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import when, regexp_replace, regexp_extract, col
import requests
from time import sleep
import random
from multiprocessing import Process
import boto3
import json
import sqlalchemy
import uuid
import os 
import timeit

### Loading data from source

In [ ]:
class AWSDBConnector:

    def __init__(self):

        self.HOST = "pinterestdbreadonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com"
        self.USER = 'project_user'
        self.PASSWORD = <password>
        self.DATABASE = 'pinterest_data'
        self.PORT = 3306
        
    def create_db_connector(self):
        engine = sqlalchemy.create_engine(f"mysql+pymysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{self.DATABASE}?charset=utf8mb4")
        return engine


new_connector = AWSDBConnector()
engine = new_connector.create_db_connector()

In [ ]:
# create uuid for unique file names
id = uuid.uuid4()
str(id)

In [ ]:
# save json file to designated location
n=0
while True:

    
    selected_row = engine.execute(f"SELECT * FROM pinterest_data LIMIT {n}, 1")
    for row in selected_row:
        result = dict(row)
    n+=1
    file_name = str(id) + "_" + str(n)+ "." + "json"
    path_to_the_file = '/Users/tanyali/Documents/practice/data/' + file_name
    file = json.dumps(result)
    with open(path_to_the_file, 'w') as outfile:
        outfile.write(file)
    # print(result)

### Initialise pyspark session and loading data

In [2]:
# initialise pyspark session
starttime = timeit.default_timer()
print("The start time is :",starttime)
spark = SparkSession.builder \
        .master(f"local[{multiprocessing.cpu_count()}]") \
        .appName("pin_app") \
        .getOrCreate()

print("The time spent is :", timeit.default_timer() - starttime)

hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoopConf.set('fs.s3a.access.key',<your access key>)
hadoopConf.set('fs.s3a.secret.key', <your secret key>)
hadoopConf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')



The start time is : 19720.038055903


23/01/11 19:07:28 WARN Utils: Your hostname, h1m1w1-CRAWLER-E10 resolves to a loopback address: 127.0.1.1; using 192.168.1.121 instead (on interface wlp0s20f3)
23/01/11 19:07:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/11 19:07:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The time spent is : 5.3080177969968645


In [3]:
# load data as json files
starttime = timeit.default_timer()
print("The start time is :",starttime)
df = spark.read.json("basicaccountstack-pinterestdataeng-proje-datalake-tcvpj2nf0cpq/*.json")
df.show(2)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19725.506774901


+----------+--------------------+----------+--------------+--------------------+-----+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+
|  category|         description|downloaded|follower_count|           image_src|index|is_image_or_video|  poster_name|       save_location|            tag_list|               title|           unique_id|
+----------+--------------------+----------+--------------+--------------------+-----+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+
|   tattoos|Elmira Kruger sha...|         1|           848|https://i.pinimg....| 8822|            image|Alicia Keller|Local save in /da...|Arm Sleeve Tattoo...|Elmira Kruger on ...|db80a4e4-293a-45c...|
|home-decor|Вместе с Русланом...|         1|           35k|https://i.pinimg....| 5996|            image|  INMYROOM.RU|Local save in /da...|Scandinavian Inte...|Где найти и как о...|4081ef0

### Data cleaning / Remove duplicates 

In [4]:
# total rows
starttime = timeit.default_timer()
print("The start time is :",starttime)
df.count()
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


The time spent is : 10.380271232999803


In [5]:
# remove duplicate rows
print("The start time is :",starttime)
df =df.dropDuplicates()
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
The time spent is : 10.451987335000013


In [6]:
# total rows after removing duplicates.
print("The start time is :",starttime)
print(df.count())
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


11154
The time spent is : 23.528107127000112


### Data transformation

In [7]:
# dataset schema
print("The start time is :",starttime)
df.printSchema()
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
root
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- downloaded: long (nullable = true)
 |-- follower_count: string (nullable = true)
 |-- image_src: string (nullable = true)
 |-- index: long (nullable = true)
 |-- is_image_or_video: string (nullable = true)
 |-- poster_name: string (nullable = true)
 |-- save_location: string (nullable = true)
 |-- tag_list: string (nullable = true)
 |-- title: string (nullable = true)
 |-- unique_id: string (nullable = true)

The time spent is : 23.612113768998825


In [8]:
print("The start time is :",starttime)
df.dtypes
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
The time spent is : 23.705660696999985


In [9]:
# column names
print("The start time is :",starttime)
df.columns
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
The time spent is : 23.780087338000158


In [10]:
# check errors in each column
print("The start time is :",starttime)
for i in df.columns:
    df.groupBy(i).count().orderBy("count", ascending=False).show()
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------+-----+
|      category|count|
+--------------+-----+
|     christmas|  999|
|       tattoos|  970|
|        quotes|  910|
|           art|  892|
|     education|  863|
|    home-decor|  861|
|        travel|  859|
|  mens-fashion|  852|
|      vehicles|  838|
|diy-and-crafts|  830|
|       finance|  786|
|event-planning|  748|
|        beauty|  746|
+--------------+-----+



+--------------------+-----+
|         description|count|
+--------------------+-----+
|No description av...|  895|
|No description av...|  182|
|Relationship Rule...|   40|
|Our mission at K&...|   24|
|Is your order tim...|   14|
|Imagine your birt...|   14|
|Top Apps for Even...|   14|
|Use our easy step...|   12|
|It doesn't matter...|   12|
|Provide a creativ...|   12|
|Macho Moda: Blog ...|    9|
|Afrocentric Bathr...|    9|
|There are more wa...|    7|
|Healing takes str...|    7|
|Mental health tat...|    7|
|Tara Jane Crandon...|    7|
|Getting a tattoo ...|    7|
|Made of quality w...|    7|
|Are you bored liv...|    6|
|Battery Powered, ...|    6|
+--------------------+-----+
only showing top 20 rows



+----------+-----+
|downloaded|count|
+----------+-----+
|         1|10823|
|         0|  331|
+----------+-----+



+--------------+-----+
|follower_count|count|
+--------------+-----+
|            2M|  375|
|            2k|  342|
|            4k|  261|
|          942k|  244|
|            5k|  235|
|            7k|  230|
|            1k|  228|
|            3k|  226|
|            5M|  215|
|            6k|  178|
|            1M|  178|
|            9k|  144|
|            8k|  132|
|           10k|  115|
|           28k|  115|
|           19k|  101|
|           20k|  100|
|           13k|   94|
|           14k|   92|
|          190k|   90|
+--------------+-----+
only showing top 20 rows



...

+--------------------+-----+
|         poster_name|count|
+--------------------+-----+
|           YourTango|  244|
|         Bored Panda|  140|
|    Our Mindful Life|   88|
|     User Info Error|   88|
|   The Minds Journal|   86|
|             Walmart|   86|
|            BuzzFeed|   81|
|Dividends Diversi...|   68|
|         Boop Decals|   60|
|         Next Luxury|   54|
|           SOCIETY19|   53|
|  One Little Project|   49|
|             Blossom|   47|
|        TheUnstitchd|   45|
|            HikenDip|   45|
|Mens Fashion - LI...|   44|
|            Wear24-7|   38|
|Mala Rock | Rock ...|   36|
|           Women.com|   35|
|       WeAreTeachers|   35|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|       save_location|count|
+--------------------+-----+
|Local save in /da...|  999|
|Local save in /da...|  970|
|Local save in /da...|  910|
|Local save in /da...|  892|
|Local save in /da...|  863|
|Local save in /da...|  861|
|Local save in /da...|  859|
|Local save in /da...|  852|
|Local save in /da...|  838|
|Local save in /da...|  830|
|Local save in /da...|  786|
|Local save in /da...|  748|
|Local save in /da...|  746|
+--------------------+-----+



+--------------------+-----+
|            tag_list|count|
+--------------------+-----+
|N,o, ,T,a,g,s, ,A...|  171|
|Stop Motion,Hobbi...|    6|
|Weekly Budget,Bud...|    3|
|Boho Chic Living ...|    2|
|Preschool Christm...|    2|
|Love My Husband Q...|    2|
|Candle Tattoo,Tre...|    2|
|Preschool Christm...|    2|
|Holiday Crafts Fo...|    2|
|Christmas Yarn,Ch...|    2|
|Christmas Art For...|    2|
|Henley Shirts,Wom...|    2|
|Printable Christm...|    2|
|Rustic Christmas ...|    2|
|Boho Kitchen,Farm...|    2|
|Budgeting Finance...|    2|
|Christmas Arts An...|    2|
|Good Vibes Quotes...|    2|
|Mode Masculine,Sh...|    2|
|Christmas Arts An...|    2|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|               title|count|
+--------------------+-----+
|No Title Data Ava...|   88|
|  Relationship Rules|   17|
|Top Apps for Even...|   14|
|DIY Abstract Hear...|   12|
|18 Daily Motivati...|   10|
|Nine Favorite Thi...|    8|
|115 Beautiful Sou...|    8|
|MULTIPLE Afrocent...|    8|
|Cobba | Non-stop ...|    7|
|55 Inspiring Ment...|    7|
|Artist Makes His ...|    7|
|40 Stunning Natur...|    7|
|51 Enigmatic Fore...|    7|
|'I Love  You' Quo...|    7|
|25 Mental Health ...|    7|
|Sunsum® Manifesta...|    6|
|Start a Party Pla...|    6|
|Giuseppe Leonardi...|    6|
|40 Elegant Dragon...|    6|
|50 Tattoo Quotes ...|    6|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|           unique_id|count|
+--------------------+-----+
|94b2a98b-ada5-4b2...|    1|
|62acf454-cb9f-409...|    1|
|c67d11ef-5f80-415...|    1|
|fde4a067-e314-4e3...|    1|
|653dda7a-b242-4a7...|    1|
|5f6cdda5-c949-44f...|    1|
|eea10ea4-65d4-49e...|    1|
|d31885b7-742a-4e2...|    1|
|d1dff8dd-8e25-498...|    1|
|9182a3ca-4831-406...|    1|
|238cfcc8-4403-4c9...|    1|
|ca29a211-4a2c-42c...|    1|
|cbbb03e5-a877-44a...|    1|
|d9be03d3-c876-45a...|    1|
|b3f03712-d2d7-45d...|    1|
|65c96357-8969-465...|    1|
|8404afec-5404-4c5...|    1|
|7e5d4399-01b3-4b8...|    1|
|9aa0c6aa-25ff-42e...|    1|
|ee9a8324-31a2-495...|    1|
+--------------------+-----+
only showing top 20 rows

The time spent is : 111.65712853500008


In [11]:
# following columns with errors.
print("The start time is :",starttime)
for i in [
 'description',
 'is_image_or_video',
 'poster_name',
 'save_location',
 'tag_list',
 'title',
 ]:
    df.groupBy(i).count().orderBy("count", ascending=False).show()
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------------+-----+
|         description|count|
+--------------------+-----+
|No description av...|  895|
|No description av...|  182|
|Relationship Rule...|   40|
|Our mission at K&...|   24|
|Is your order tim...|   14|
|Imagine your birt...|   14|
|Top Apps for Even...|   14|
|Use our easy step...|   12|
|It doesn't matter...|   12|
|Provide a creativ...|   12|
|Macho Moda: Blog ...|    9|
|Afrocentric Bathr...|    9|
|There are more wa...|    7|
|Healing takes str...|    7|
|Mental health tat...|    7|
|Tara Jane Crandon...|    7|
|Getting a tattoo ...|    7|
|Made of quality w...|    7|
|Are you bored liv...|    6|
|Battery Powered, ...|    6|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|   is_image_or_video|count|
+--------------------+-----+
|               image| 9920|
|multi-video(story...|  895|
|               video|  332|
|                null|    7|
+--------------------+-----+



+--------------------+-----+
|         poster_name|count|
+--------------------+-----+
|           YourTango|  244|
|         Bored Panda|  140|
|    Our Mindful Life|   88|
|     User Info Error|   88|
|   The Minds Journal|   86|
|             Walmart|   86|
|            BuzzFeed|   81|
|Dividends Diversi...|   68|
|         Boop Decals|   60|
|         Next Luxury|   54|
|           SOCIETY19|   53|
|  One Little Project|   49|
|             Blossom|   47|
|        TheUnstitchd|   45|
|            HikenDip|   45|
|Mens Fashion - LI...|   44|
|            Wear24-7|   38|
|Mala Rock | Rock ...|   36|
|           Women.com|   35|
|       WeAreTeachers|   35|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|       save_location|count|
+--------------------+-----+
|Local save in /da...|  999|
|Local save in /da...|  970|
|Local save in /da...|  910|
|Local save in /da...|  892|
|Local save in /da...|  863|
|Local save in /da...|  861|
|Local save in /da...|  859|
|Local save in /da...|  852|
|Local save in /da...|  838|
|Local save in /da...|  830|
|Local save in /da...|  786|
|Local save in /da...|  748|
|Local save in /da...|  746|
+--------------------+-----+



+--------------------+-----+
|            tag_list|count|
+--------------------+-----+
|N,o, ,T,a,g,s, ,A...|  171|
|Stop Motion,Hobbi...|    6|
|Weekly Budget,Bud...|    3|
|Boho Chic Living ...|    2|
|Preschool Christm...|    2|
|Love My Husband Q...|    2|
|Candle Tattoo,Tre...|    2|
|Preschool Christm...|    2|
|Holiday Crafts Fo...|    2|
|Christmas Yarn,Ch...|    2|
|Christmas Art For...|    2|
|Henley Shirts,Wom...|    2|
|Printable Christm...|    2|
|Rustic Christmas ...|    2|
|Boho Kitchen,Farm...|    2|
|Budgeting Finance...|    2|
|Christmas Arts An...|    2|
|Good Vibes Quotes...|    2|
|Mode Masculine,Sh...|    2|
|Christmas Arts An...|    2|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|               title|count|
+--------------------+-----+
|No Title Data Ava...|   88|
|  Relationship Rules|   17|
|Top Apps for Even...|   14|
|DIY Abstract Hear...|   12|
|18 Daily Motivati...|   10|
|Nine Favorite Thi...|    8|
|115 Beautiful Sou...|    8|
|MULTIPLE Afrocent...|    8|
|Cobba | Non-stop ...|    7|
|55 Inspiring Ment...|    7|
|Artist Makes His ...|    7|
|40 Stunning Natur...|    7|
|51 Enigmatic Fore...|    7|
|'I Love  You' Quo...|    7|
|25 Mental Health ...|    7|
|Sunsum® Manifesta...|    6|
|Start a Party Pla...|    6|
|Giuseppe Leonardi...|    6|
|40 Elegant Dragon...|    6|
|50 Tattoo Quotes ...|    6|
+--------------------+-----+
only showing top 20 rows

The time spent is : 146.47906401900036


In [12]:
# replace error value or empty value with None
print("The start time is :",starttime)
df = df.replace({'No description available Story format': None}, subset = ['description'])\
       .replace({'No description available': None}, subset = ['description'])

df.groupBy('description').count().orderBy("count", ascending=False).show(3)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------------+-----+
|         description|count|
+--------------------+-----+
|                null| 1077|
|Relationship Rule...|   40|
|Our mission at K&...|   24|
+--------------------+-----+
only showing top 3 rows

The time spent is : 155.81028705800054


In [13]:
# display error values in 'poster_name' column
print("The start time is :",starttime)
df.groupBy('poster_name').count().orderBy("count", ascending=False).show(5)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+----------------+-----+
|     poster_name|count|
+----------------+-----+
|       YourTango|  244|
|     Bored Panda|  140|
|Our Mindful Life|   88|
| User Info Error|   88|
|         Walmart|   86|
+----------------+-----+
only showing top 5 rows

The time spent is : 163.6753728799995


In [14]:
# replace error values "User Info Error" in column 'poster_name' with null
print("The start time is :",starttime)
df = df.replace({'User Info Error': None}, subset = ['poster_name'])
df.groupBy('poster_name').count().orderBy("count", ascending=False).show(4)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+----------------+-----+
|     poster_name|count|
+----------------+-----+
|       YourTango|  244|
|     Bored Panda|  140|
|            null|   88|
|Our Mindful Life|   88|
+----------------+-----+
only showing top 4 rows

The time spent is : 170.98469567400025


In [15]:
# display error values in 'tag_list' column
print("The start time is :",starttime)
df.groupBy('tag_list').count().orderBy("count", ascending=False).show(5)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------------+-----+
|            tag_list|count|
+--------------------+-----+
|N,o, ,T,a,g,s, ,A...|  171|
|Stop Motion,Hobbi...|    6|
|Weekly Budget,Bud...|    3|
|Love My Husband Q...|    2|
|Preschool Christm...|    2|
+--------------------+-----+
only showing top 5 rows

The time spent is : 178.5500242929993


In [16]:
# get error value in tag_list
print("The start time is :",starttime)
tag = df.groupBy('tag_list').count().orderBy("count", ascending=False)
print(tag.collect()[0][0])
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e
The time spent is : 188.3994297019999


In [17]:
# replace error values with null
print("The start time is :",starttime)
df = df.replace({'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e': None}, subset = ['tag_list'])
# display tag list values after remove error values
df.groupBy('tag_list').count().orderBy("count", ascending=False).show(4)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------------+-----+
|            tag_list|count|
+--------------------+-----+
|                null|  171|
|Stop Motion,Hobbi...|    6|
|Weekly Budget,Bud...|    3|
|Love My Husband Q...|    2|
+--------------------+-----+
only showing top 4 rows

The time spent is : 197.63270826200096


In [18]:
# display error values in 'title' column
print("The start time is :",starttime)
df.groupBy('title').count().orderBy("count", ascending=False).show(5)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------------+-----+
|               title|count|
+--------------------+-----+
|No Title Data Ava...|   88|
|  Relationship Rules|   17|
|Top Apps for Even...|   14|
|DIY Abstract Hear...|   12|
|18 Daily Motivati...|   10|
+--------------------+-----+
only showing top 5 rows

The time spent is : 205.35657458599962


In [19]:
# get error value in title column
print("The start time is :",starttime)
tag = df.groupBy('title').count().orderBy("count", ascending=False)
print(tag.collect()[0][0])
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


No Title Data Available
The time spent is : 214.15646095899865


In [20]:
# replace error value with null
print("The start time is :",starttime)
df = df.replace({'No Title Data Available': None}, subset = ['title'])
df.groupBy('title').count().orderBy("count", ascending=False).show(4)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------------+-----+
|               title|count|
+--------------------+-----+
|                null|   88|
|  Relationship Rules|   17|
|Top Apps for Even...|   14|
|DIY Abstract Hear...|   12|
+--------------------+-----+
only showing top 4 rows

The time spent is : 222.5112628800016


In [21]:
# display error value in save_location column
print("The start time is :",starttime)
df.groupBy('save_location').count().orderBy("count", ascending=False).show(50)
print("The time spent is :", timeit.default_timer() - starttime)



The start time is : 19785.316717128


+--------------------+-----+
|       save_location|count|
+--------------------+-----+
|Local save in /da...|  999|
|Local save in /da...|  970|
|Local save in /da...|  910|
|Local save in /da...|  892|
|Local save in /da...|  863|
|Local save in /da...|  861|
|Local save in /da...|  859|
|Local save in /da...|  852|
|Local save in /da...|  838|
|Local save in /da...|  830|
|Local save in /da...|  786|
|Local save in /da...|  748|
|Local save in /da...|  746|
+--------------------+-----+

The time spent is : 229.30968092799958


In [22]:
# get example of error value in save_location column
print("The start time is :",starttime)
location_column = df.groupBy('save_location').count().orderBy("count", ascending=False)
print(location_column.collect()[0][0])
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


Local save in /data/christmas
The time spent is : 236.12930186700032


In [23]:
# replace/remove error part of the values 
print("The start time is :",starttime)
df = df.withColumn('save_location', regexp_replace('save_location', 'Local save in ', '')) 
df.groupBy('save_location').count().orderBy("count", ascending=False).show()
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------------+-----+
|       save_location|count|
+--------------------+-----+
|     /data/christmas|  999|
|       /data/tattoos|  970|
|        /data/quotes|  910|
|           /data/art|  892|
|     /data/education|  863|
|    /data/home-decor|  861|
|        /data/travel|  859|
|  /data/mens-fashion|  852|
|      /data/vehicles|  838|
|/data/diy-and-crafts|  830|
|       /data/finance|  786|
|/data/event-planning|  748|
|        /data/beauty|  746|
+--------------------+-----+

The time spent is : 245.30185736299973


In [24]:
# display error and null value
print("The start time is :",starttime)
df.groupBy('image_src').count().orderBy("count", ascending=False).show(4)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------------+-----+
|           image_src|count|
+--------------------+-----+
|    Image src error.|  323|
|                null|    7|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
+--------------------+-----+
only showing top 4 rows

The time spent is : 256.0605765469991


In [25]:
# replace error values with null
print("The start time is :",starttime)
df = df.replace({'Image src error.': None}, subset = ['image_src'])
# display image_src values
df.groupBy('image_src').count().orderBy("count", ascending=False).show(4)
print("The time spent is :", timeit.default_timer() - starttime)



The start time is : 19785.316717128


+--------------------+-----+
|           image_src|count|
+--------------------+-----+
|                null|  330|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
+--------------------+-----+
only showing top 4 rows

The time spent is : 264.08887307999976


In [26]:
# drop the rows with null values in 'image_src' column, as we don't want a row without a pin
print("The start time is :",starttime)
df.groupBy('image_src').count().orderBy("count", ascending=False).show(5)
print("The time spent is :", timeit.default_timer() - starttime)
df = df.na.drop(subset=["image_src"])
# display image_src values after removing nul values



The start time is : 19785.316717128


+--------------------+-----+
|           image_src|count|
+--------------------+-----+
|                null|  330|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
+--------------------+-----+
only showing top 5 rows

The time spent is : 272.36532181800067


In [27]:
# check errors in column
print("The start time is :",starttime)
df.groupBy('follower_count').count().orderBy("count", ascending=False).show()
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------+-----+
|follower_count|count|
+--------------+-----+
|            2M|  374|
|            2k|  325|
|            4k|  250|
|          942k|  244|
|            5k|  232|
|            7k|  227|
|            1k|  219|
|            3k|  219|
|            5M|  215|
|            1M|  178|
|            6k|  172|
|            9k|  141|
|            8k|  126|
|           28k|  115|
|           10k|  113|
|           20k|   99|
|           19k|   93|
|          190k|   90|
|           13k|   90|
|           11k|   88|
+--------------+-----+
only showing top 20 rows

The time spent is : 282.495029144


In [28]:
# create a pandas dataframe which run much faster
print("The start time is :",starttime)
pddf = df.toPandas()
# check if there is any characters in the follower_count column
a = []
for i in pddf['follower_count'].to_list():
    if i is not None:
        for char in i:
            if  not char.isdigit():
                a.append(char)
print("number of character is: ", len(a))
print(set(a))      
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


number of character is:  9001
{'n', 's', 'r', ' ', 'f', 'U', 'k', 'E', 'o', 'I', 'M', 'e'}
The time spent is : 292.55941720600094


In [29]:
# check each character 
print("The start time is :",starttime)
for i in pddf['follower_count'].to_list():
    if 'E' in i:
        print(i)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
User Info Error
The time spent is : 292.628748471001


In [30]:
# replace 'User Info Error' with 0
print("The start time is :",starttime)
df = df.replace({'User Info Error': "0"}, subset = ['follower_count'])
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
The time spent is : 292.74224410399984


In [31]:
# check it again if there is any characters in the follower_count column
print("The start time is :",starttime)
pddf = df.toPandas()
a = []
for i in pddf['follower_count'].to_list():
    if i is not None:
        for char in i:
            if  not char.isdigit():
                a.append(char)
print("number of 'M' or 'k' is: ", len(a))
print(set(a))  
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


number of 'M' or 'k' is:  8986
{'k', 'M'}
The time spent is : 300.9085890319984


In [32]:
# replaced 'M', 'k' by '000000' and '000'
print("The start time is :",starttime)
df = df.withColumn('follower_count', 
    when(df.follower_count.endswith('M'),regexp_replace(df.follower_count,'M','000000')) \
   .when(df.follower_count.endswith('k'),regexp_replace(df.follower_count,'k','000')) \
   .otherwise(df.follower_count)) 
print("The time spent is :", timeit.default_timer() - starttime)



The start time is : 19785.316717128
The time spent is : 301.0769321730004


In [33]:
# check it again
print("The start time is :",starttime)
df.groupBy('follower_count').count().orderBy("count", ascending=False).show()
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+--------------+-----+
|follower_count|count|
+--------------+-----+
|       2000000|  374|
|          2000|  325|
|          4000|  250|
|        942000|  244|
|          5000|  232|
|          7000|  227|
|          3000|  219|
|          1000|  219|
|       5000000|  215|
|       1000000|  178|
|          6000|  172|
|          9000|  141|
|          8000|  126|
|         28000|  115|
|         10000|  113|
|         20000|   99|
|         19000|   93|
|         13000|   90|
|        190000|   90|
|         11000|   88|
+--------------+-----+
only showing top 20 rows

The time spent is : 308.90140081700156


In [34]:
# cast follower_count column type as int
print("The start time is :",starttime)
import pyspark.sql.functions as F
df = df.withColumn("follower_count", F.col("follower_count").cast("int"))
df.show(2)
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128


+----------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+----------------+--------------------+---------------------+--------------------+
|  category|         description|downloaded|follower_count|           image_src|index|is_image_or_video|         poster_name|   save_location|            tag_list|                title|           unique_id|
+----------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+----------------+--------------------+---------------------+--------------------+
|   tattoos|Elmira Kruger sha...|         1|           848|https://i.pinimg....| 8822|            image|       Alicia Keller|   /data/tattoos|Arm Sleeve Tattoo...| Elmira Kruger on ...|db80a4e4-293a-45c...|
|home-decor|И хоть у шведов л...|         1|        136000|https://i.pinimg....| 6447|            image|PUFIK Interiors &...|/data/home-decor|Cheap Home Decor,...|〚 Warm na

In [35]:
# cassandra not allow use index as a key name, change "index" to "idx"


print("The start time is :",starttime)
df = df.withColumnRenamed('index', 'idx')

print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
The time spent is : 316.4588153699988


In [36]:
# reorder selected columns
print("The start time is :",starttime)
df = df.select('idx', 'title', 'poster_name', 'category', 'follower_count', 'description', 'image_src', 'is_image_or_video', 'tag_list', 'unique_id')
print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
The time spent is : 316.5580150880014


In [37]:
# reorder dataset with acsending index
print("The start time is :",starttime)
df = df.orderBy("idx", ascending=True)

print("The time spent is :", timeit.default_timer() - starttime)


The start time is : 19785.316717128
The time spent is : 316.62473346299885


In [38]:
# schema = StructType([
#     StructField("index",IntegerType(),True), \
#     StructField("title",StringType(),True), \
#     StructField("poster_name",StringType(),True), \
#     StructField("category", StringType(), True), \
#     StructField("follower_count", IntegerType(), True), \
#     StructField("description", StringType(), True), \
#     StructField("downloaded",IntegerType(),True), \
#     StructField("image_src",StringType(),True), \
#     StructField("is_image_or_video", StringType(), True), \
#     StructField("save_location", StringType(), True), \
#     StructField("tag_list", StringType(), True),  \
#     StructField("unique_id",StringType(),True), \
#         ])
# df = spark.createDataFrame(data=df.collect(),schema=schema)

### Send data to Cassandra

In [39]:
# initialise cassandra driver
cluster = Cluster()
session = cluster.connect()

In [ ]:
# create a cassandra keyspace
session.execute("CREATE KEYSPACE pinterest_project WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3};")

In [41]:
# initialise named cassandra key space
try:
    session.execute("USE pinterest_project;")
except:
    pass

In [42]:
try:
    session.execute("DROP TABLE pinterest")
except:
    pass

In [43]:
session.execute("CREATE TABLE pinterest(idx int PRIMARY KEY, title text, poster_name text, category text, follower_count int, description text, image_src text, is_image_or_video text, tag_list text, unique_id text);")


In [44]:
# make preparedUpdate statements
print("The start time is :",starttime)
preparedUpdate = session.prepare(
    """ 
    INSERT INTO pinterest (idx, title, poster_name, category, follower_count, description, image_src, is_image_or_video, tag_list, unique_id) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """
)


# write df to cassandra
for item in df.collect():
    session.execute(preparedUpdate, [item[0], item[1], item[2], item[3], item[4], item[5], item[6], item[7], item[8], item[9]])

print("The time spent is :", timeit.default_timer() - starttime)



The start time is : 19785.316717128


The time spent is : 367.31381211199914


In [45]:
# Read pinterest table in cassandra
results = session.execute("SELECT * FROM pinterest")

In [46]:
# check if table in cassandra has read
# print first row of the table
print(results[0])

/tmp/ipykernel_207893/3363985775.py:3: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  print(results[0])


Row(idx=4317, category='education', description='Women in the Modern World : Annals of the American Academy of Political and Social Science, V143, May, 1929', follower_count=2000000, image_src='https://i.pinimg.com/originals/d9/85/5e/d9855ea66daf142e7a8e8d3671dfe1af.jpg', is_image_or_video='image', poster_name='Walmart', tag_list='Way Of Life,The Life,Inspiring Meaning,John Calvin,Battle Of Waterloo,National Convention,Move Mountains,Public Speaking,Human Nature', title='Women in the Modern World: Annals of the American Academy of Political and Social Science, V143, May, 1929 (Paperback) - Walmart.com', unique_id='ebce1b3d-149c-498d-bef6-e5538dc4d341')


In [47]:
# shutdown cassandra session and cluster

session.cluster.shutdown()
session.shutdown()

In [48]:
# stop spark
spark.stop()